## **Tela de gerenciamento dos Jogos**

#### **Importação das Bibliotecas Necessárias**

- **`os`**: Para acessar variáveis de ambiente.
- **`load_dotenv`**: Para carregar variáveis do arquivo `.env`.
- **`pandas`**: Para manipulação e análise de dados.
- **`psycopg2`**: Para conexão e operações com o banco de dados PostgreSQL.
  - **`OperationalError`**: Para tratar erros operacionais do banco de dados.
  - **`IntegrityError`**: Para capturar violações de integridade.
- **`sqlalchemy`**: Para abstração e gerenciamento de conexões de banco de dados.
  - **`create_engine`**: Para criar uma conexão com o banco de dados.
- **`panel`**: Para criar interfaces e dashboards interativos.

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, IntegrityError
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn

#### **Carregamento e Acesso às Variáveis de Ambiente**

- **`load_dotenv()`**: Carrega as variáveis de ambiente a partir do arquivo `.env`.
- **`db_name`**: Obtém o nome do banco de dados.
- **`db_user`**: Obtém o nome de usuário para a conexão com o banco de dados.
- **`db_password`**: Obtém a senha para a conexão com o banco de dados.
- **`db_host`**: Obtém o endereço do host do banco de dados.
- **`db_port`**: Obtém o número da porta para a conexão com o banco de dados.
- **`database_url`**: Obtém a URL completa para a conexão com o banco de dados.

In [ ]:
load_dotenv()

db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

database_url = os.getenv('DATABASE_URL')

#### **Estabelecimento de Conexão com o Banco de Dados e Criação da Engine**

1. **Conexão com `psycopg2`**:
   - **Objetivo**: Tentar conectar ao banco de dados PostgreSQL usando as variáveis de ambiente.
   - **Parâmetros**:
     - **`dbname`**: Nome do banco de dados.
     - **`user`**: Nome de usuário.
     - **`password`**: Senha do usuário.
     - **`host`**: Endereço do servidor.
     - **`port`**: Número da porta.
   - **Sucesso**: Exibe uma mensagem de confirmação se a conexão for estabelecida com sucesso.
   - **Erro**: Se ocorrer um `OperationalError`, exibe uma mensagem com o motivo da falha.

2. **Criação da Engine com `sqlalchemy`**:
   - **Objetivo**: Criar uma engine de conexão para o banco de dados usando a URL completa.
   - **Parâmetro**:
     - **`database_url`**: URL completa de conexão com o banco de dados.
   - **Função**: A engine abstrai a conexão e facilita a execução de operações e consultas no banco de dados.

In [ ]:
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Conexão estabelecida com sucesso!")
    
except OperationalError as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


engine = create_engine(database_url)

#### **Inicialização da Extensão do Panel**

- **`pn.extension()`**: Inicializa as extensões do Panel necessárias para o funcionamento dos widgets e componentes.

- **`pn.extension('tabulator')`**: Carrega a extensão `Tabulator` do Panel, que permite a criação de tabelas interativas e altamente configuráveis.

- **`pn.extension(notifications=True)`**: Ativa o sistema de notificações do Panel, permitindo exibir mensagens de sucesso, erro ou alerta para o usuário.

In [ ]:
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

#### **Definição de Widgets com Panel**

- **Campos de Texto**:
  - **`game_name`**: Para o nome do jogo.
  - **`type`**: Para o tipo do jogo.
  - **`description`**: Para a descrição do jogo.

- **Botões**:
  - **`buttonSelect`**: Consultar.
  - **`buttonInsert`**: Inserir.
  - **`buttonDelete`**: Excluir.

In [10]:
flag = ""

game_name = pn.widgets.TextInput(
    name="Nome do Jogo",
    value="",
    placeholder="Digite o nome do jogo",
    disabled=False
)

type = pn.widgets.TextInput(
    name="Tipo",
    value="",
    placeholder="Digite o tipo do jogo",
    disabled=False
)

description = pn.widgets.TextInput(
    name="Descrição",
    value="",
    placeholder="Digite a descrição do jogo",
    disabled=False
)

buttonSelect = pn.widgets.Button(name="Consultar", button_type="default")
buttonInsert = pn.widgets.Button(name="Inserir", button_type="default")
buttonDelete = pn.widgets.Button(name="Excluir", button_type="default")

#### **Funções para Manipulação do Banco de Dados**

- **`query_all()`**:
  - **Objetivo**: Recupera todos os registros da tabela `bd_jogos.jogo` e exibe em uma tabela interativa.
  - **Retorno**: Tabela `Tabulator` com todos os jogos.

- **`select_game()`**:
  - **Objetivo**: Consulta registros na tabela `bd_jogos.jogo` com base no nome ou tipo do jogo fornecido.
  - **Tratamento de Erros**: Exibe um alerta se a consulta falhar.

- **`insert_game()`**:
  - **Objetivo**: Insere um novo jogo na tabela `bd_jogos.jogo` com base nas informações fornecidas.
  - **Tratamento de Erros**: 
    - Exibe um alerta se o nome do jogo já existir.
    - Reverte a transação e exibe uma mensagem de erro se a inserção falhar.

- **`delete_game()`**:
  - **Objetivo**: Exclui um jogo da tabela `bd_jogos.jogo` com base no nome fornecido.
  - **Tratamento de Erros**: 
    - Reverte a transação e exibe uma mensagem de erro se a exclusão falhar.

In [11]:
def query_all():
    query = f"SELECT * FROM bd_jogos.jogo"
    df = pd.read_sql_query(query, conn)
    return pn.widgets.Tabulator(df)

def select_game():
    try:
        query = f"SELECT * FROM bd_jogos.jogo WHERE ('{game_name.value_input}'='{flag}' and '{type.value_input}'  = '{flag}') or nome = '{game_name.value_input}' or tipo = '{type.value_input}'"
        df = pd.read_sql(query, con=engine)
        table = pn.widgets.Tabulator(df, layout='fit_data')
        return table
    except Exception as e:
        return pn.pane.Alert('Não foi possível consultar!')

def insert_game():
    try:
        cursor = conn.cursor()
        query = f"insert into bd_jogos.jogo (nome, tipo, descricao) values (%s, %s, %s)"

        cursor.execute(query, (game_name.value_input, type.value_input, description.value_input))

        conn.commit()
        return query_all()
    except IntegrityError as e:
        if e.pgcode == "23505":
            return pn.pane.Alert("O nome do jogo já existe. Por favor, escolha um nome diferente.")
        else:
            conn.rollback()
            cursor.close()
            return pn.pane.alert("Não foi possível inserir!")
    except Exception as e:
        conn.rollback()
        cursor.close()
        return pn.pane.Alert("Não foi possível inserir!")

def delete_game():
    try:
        cursor = conn.cursor()
        query = f"DELETE FROM bd_jogos.jogo WHERE jogo.nome = %s"
        
        cursor.execute(query, (game_name.value_input,))

        conn.commit()

        return query_all()
    except Exception as e:
        print(e)
        conn.rollback()
        cursor.close()
        return pn.pane.Alert("Não foi possível excluir!")

#### **Função para Criar Tabelas Interativas**

- **`table_creator(cons, ins, exc)`**:
  - **Objetivo**: Determina qual função executar com base nos parâmetros fornecidos.
  - **Parâmetros**:
    - **`cons`**: Se verdadeiro, chama `select_game()`.
    - **`ins`**: Se verdadeiro, chama `insert_game()`.
    - **`exc`**: Se verdadeiro, chama `delete_game()`.

- **`interactive_table`**:
  - **Objetivo**: Cria uma tabela interativa vinculada aos botões `buttonSelect`, `buttonInsert`, e `buttonDelete`.
  - **Função**: Utiliza `pn.bind()` para associar a função `table_creator` aos botões, permitindo a execução da função apropriada quando um botão é clicado.

In [12]:
def table_creator(cons, ins, exc):
    if cons:
        return select_game()
    if ins:
        return insert_game()
    if exc:
        return delete_game()

interactive_table = pn.bind(table_creator, buttonSelect, buttonInsert, buttonDelete)

#### **Layout da Interface**

- **Organização**:
  - **Coluna 1**: 
    - Título: "CRUD de jogos"
    - Campos para entrada de dados (nome, tipo, descrição)
    - Botões para consultar, inserir e excluir jogos
  - **Coluna 2**: 
    - Tabela interativa exibindo os resultados das operações

- **Método**: 
  - **`.servable()`**: Torna a interface interativa e visível em um servidor Panel.

In [13]:
pn.Row(pn.Column("Tela de jogos",
    game_name,
    type,
    description,
    pn.Row(buttonSelect),
    pn.Row(buttonInsert),
    pn.Row(buttonDelete)),
    pn.Column(interactive_table)).servable()